# Read a binary ats file (old ats file)

In [1]:
import json
import sys, os
import struct
sys.path.append(os.path.realpath('../../include'))
import ats_base as ats
import json_header as jh


ModuleNotFoundError: No module named 'pycstruct'

In [ ]:
# take an existing file 084_V01_C02_R001_THx_BL_2S.ats
#base = '084_V01_C02_R001_THx_BL_2S'
base = '084_V01_C00_R001_TEx_BL_2S'
ats_file = base + '.ats'              # the existing
atss_file = base + '.atss'            # new as double, scaled for E!

ats_header, slice_headers = ats.read_atsheader(ats_file)     # read the binary header
print("old ats header data:")
print(json.dumps(ats_header, indent=2, sort_keys=False, ensure_ascii=False))  # check the result of binary data
       

old ats header data:
{
  "header_length": 1024,
  "header_version": 75,
  "samples": 31716,
  "sample_rate": 0.5,
  "start": 1250774545,
  "lsbval": 5.6232e-05,
  "GMToffset": 0,
  "orig_sample_rate": 0.0,
  "serial_number": 84,
  "serial_number_ADC_board": 56410,
  "channel_number": 0,
  "chopper": 1,
  "channel_type": "Ex",
  "sensor_type": "EFP06",
  "sensor_serial_number": 0,
  "x1": -30.0,
  "y1": 0.0,
  "z1": 0.0,
  "x2": 30.0,
  "y2": 0.0,
  "z2": 0.0,
  "dipole_length": 0.0,
  "angle": 0.0,
  "rho_probe_ohm": 572.3670043945312,
  "DC_offset_voltage_mV": 0.004233600106090307,
  "gain_stage1": 8.0,
  "gain_stage2": 0.0,
  "iLat_ms": 140494308,
  "iLong_ms": 104846232,
  "iElev_cm": 108831,
  "Lat_Long_TYPE": "G",
  "coordinate_type": "",
  "ref_meridian": 0,
  "Northing": 0.0,
  "Easting": 0.0,
  "gps_clock_status": "C",
  "GPS_accuracy": 0,
  "offset_UTC": 0,
  "SystemType": "ADU07",
  "survey_header_filename": "Reference si",
  "type_of_meas": "MT",
  "DCOffsetCorrValue": 0.0,


In [ ]:
# change the site name ... this old site has none
ats_header['SiteName'] = "Sarıçam"                           
ats_header['Operator'] = "newbee" 
# test success in old binary format
testfile = 'testheader.ats'
ats.write_atsheader(ats_header, testfile, False)    # with atsedit you can prove the header and should see the new site name

# import will convert lat lon elev and so on
channel = jh.channel_form_oldheader(ats_header)     # copy old into new header (with site name)

print("channel imported from ats file:")

print(json.dumps(channel, indent=2, sort_keys=False, ensure_ascii=False))


channel imported from ats file:
{
  "serial": 84,
  "system": "ADU-07",
  "channel_no": 0,
  "run": 1,
  "channel_type": "Ex",
  "sample_rate": 0.5,
  "date": "2009-08-20",
  "time": "13:22:25",
  "fracs": 0.0,
  "latitude": 39.026196666666664,
  "longitude": 29.123953333333333,
  "elevation": 1088.31,
  "angle": 0.0,
  "dip": 90.0,
  "units": "mV",
  "source": "",
  "site": "Sarıçam",
  "sensor_calibration": {
    "sensor": "EFP-06",
    "serial": 0,
    "chopper": 1,
    "units_amplitude": "mV",
    "units_frequency": "Hz",
    "units_phase": "degrees",
    "date": "1970-01-01",
    "time": "00:00:00",
    "Operator": "",
    "f": [
      0.0
    ],
    "a": [
      0.0
    ],
    "p": [
      0.0
    ]
  },
  "dipole_length": 60.0,
  "lsb": 5.6232e-05,
  "samples": 31716
}


In [ ]:
# the new file REQUIRES calibrtation data INSIDE the header
# ALL what is needed for processing must be inside the header
# for the old data from Sarıçam calibration is lost
# take a default master function (done here)
# or take a real calibration from the database
# if f, a, p are 0.0 the software will calculate a calibration
#    from sensor and chopper
# this is the case for boards, filters or fluxgates

# serial indicates take calibration from master calibration data file

channel['sensor_calibration']['f'], channel['sensor_calibration']['a'], channel['sensor_calibration']['p'] = jh.get_cal_from_db("", channel['sensor_calibration']['sensor'], 0, 1)


print(json.dumps(channel['sensor_calibration'], indent=2, sort_keys=False, ensure_ascii=False))

no such table: EFP-06
{
  "sensor": "EFP-06",
  "serial": 0,
  "chopper": 1,
  "units_amplitude": "mV",
  "units_frequency": "Hz",
  "units_phase": "degrees",
  "date": "1970-01-01",
  "time": "00:00:00",
  "Operator": "",
  "f": [],
  "a": [],
  "p": []
}


In [ ]:
# now FIRST WRITE THE ATSS file with doubles
# ATSS wants to SCALE the E field in case
# watch that the function returns a NEW units in case

channel['units'] = ats.atss_file_from_atsfile(ats_file, channel, jh.atss_filename(channel) + ".atss")

# and write the JSON file (after possible scaling to E = mV/km )

jh.to_json(channel)




write:  084_ADU-07_C000_R001_TEx_2s.json
